In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


# **Importing Dataset**

In [ ]:
dataset1 = pd.read_csv('/drive/My Drive/Colab Notebooks/delivery_orders_march.csv')
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   orderid              int64  
 1   pick                 int64  
 2   1st_deliver_attempt  float64
 3   2nd_deliver_attempt  float64
 4   buyeraddress         object 
 5   selleraddress        object 
dtypes: float64(2), int64(2), object(2)
memory usage: 145.4+ MB


# **Calculate timestamp**

In [ ]:
#converting the timestamp type into integer
dataset['1st_deliver_attempt'] = dataset['1st_deliver_attempt'].astype('int64')
dataset['2nd_deliver_attempt'] = dataset['2nd_deliver_attempt'].fillna(0)
dataset['2nd_deliver_attempt'] = dataset['2nd_deliver_attempt'].astype('int64')

In [ ]:
#convert time sec
dataset['pick'] = (dataset['pick'] + (3600*8))/(3600*24)
dataset['1st_deliver_attempt'] = (dataset['1st_deliver_attempt'] + (3600*8))/(3600*24)
dataset['2nd_deliver_attempt'] = (dataset['2nd_deliver_attempt'] + (3600*8))/(3600*24)
dataset['2nd_deliver_attempt'] = dataset['2nd_deliver_attempt'].apply(lambda x: 0 if x<1 else x)

In [ ]:
#converting time to day
t1 = dataset['pick'].values.astype('datetime64[D]')
t2 = dataset['1st_deliver_attempt'].values.astype('datetime64[D]')
t3 = dataset['2nd_deliver_attempt'].values.astype('datetime64[D]')

In [ ]:
#counting the day for sending the order
dataset['Order 1'] = np.busday_count(t1,t2,'1111110',holidays=['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])
dataset['Order 2'] = np.busday_count(t2,t3,'1111110',holidays=['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])

In [ ]:
#converting non 2nd delivery day
dataset['Order 2'] = dataset['Order 2'].apply(lambda x: 0 if x<1 else x)

# **Converting Address**

In [ ]:
sla_matrix_1st_attempt = [
    [3, 5, 7, 7],
    [5, 5, 7, 7],
    [7, 7, 7, 7],
    [7, 7, 7, 7],
]
locations = ["Metro Manila", "Luzon", "Visayas", "Mindanao"]
locations = [loc.lower() for loc in locations]
min_length = min(map(len, locations))
trunc_location_to_index = {loc[-min_length:]: i for i, loc in enumerate(locations)}

In [ ]:
# convert address to index
dataset['buyeraddress'] = dataset['buyeraddress'].apply(lambda s: s[-min_length:].lower()).map(trunc_location_to_index)
dataset['selleraddress'] = dataset['selleraddress'].apply(lambda s: s[-min_length:].lower()).map(trunc_location_to_index)

# **Calculating SLA Based Delayed Order**

In [ ]:
Order1 = np.array(dataset['Order 1'])
Order2 = np.array(dataset['Order 2'])
Index1 = np.array(dataset['buyeraddress'])
Index2 = np.array(dataset['selleraddress'])

In [ ]:
orderlate = []
for j in range(len(Index1)):
  if Order2[j] > 0:
    if Order2[j] > 3 :
      orderlate.append(1)
    else:
      orderlate.append(0)
  else:
    Index_1 = Index1[j]
    Index_2 = Index2[j]
    threshold = sla_matrix_1st_attempt[Index_1][Index_2]
    if Order1[j] > threshold:
      orderlate.append(1)
    else:
      orderlate.append(0)

# **Saving The File**

In [ ]:
result = pd.DataFrame({'orderid':dataset['orderid'], 'is_late': orderlate})
result.to_csv('/drive/My Drive/Colab Notebooks/Logis1.csv', index=False)

# **Checking the answer**

In [ ]:
dataset.tail()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,Order 1,Order 2
3176308,31504087640510,18354.751481,18356.557199,0.000000,0,0,2,0
3176309,31504147352227,18355.599664,18358.887118,18359.494572,0,0,2,1
3176310,31504462290482,18354.811933,18356.413657,0.000000,0,0,2,0
3176311,31504851495943,18355.555486,18359.471667,0.000000,1,0,3,0
3176312,31505199116340,18354.776887,18358.584977,18359.484259,1,0,3,1


In [ ]:
result.tail()

,orderid,is_late
3176308,31504087640510,0
3176309,31504147352227,0
3176310,31504462290482,0
3176311,31504851495943,0
3176312,31505199116340,0


5 last entry show that there are no delayed order, thus is_late status for 5 entries in the result is 0